<a href="https://colab.research.google.com/github/paulo037/text2sqsl/blob/main/preprocessing/SpiderDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets langchain_community

In [ ]:
import sqlite3
import sqlparse
from datasets import load_dataset, Dataset
from langchain_community.utilities.sql_database import SQLDatabase

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##### Funcoes Auxiliares

In [ ]:
def get_db(db_path):
  connection_string="sqlite:///"+db_path
  return SQLDatabase.from_uri(connection_string, sample_rows_in_table_info=0)

def get_schema(db_path, qtd_ex=0):
  connection_string="sqlite:///"+db_path
  db = SQLDatabase.from_uri(connection_string, sample_rows_in_table_info=qtd_ex)
  return str(db.table_info)

In [ ]:
def run_db(db, query: str):
    try:
        answer = db.run(query, include_columns=True)
        if answer:
          return eval(answer)
        else:
          return [{'Nenhum':'Nenhum resultado encontrado'}]
    except Exception as e:
        return [{'Error' : f"{str(e)}"}]

In [ ]:
import json
def informacoes_json(json_path, db_path, dbs,dbsComEx, db_ids_com_inconsistencia):

  with open(json_path) as f:
      data = json.load(f)

  dataset_dict = {'db_id':[], 'schema':[],'schemaComEx':[], 'question':[], 'query':[], 'answer':[], 'complexity':[], 'distinct':[], 'like':[], 'between':[], 'order_by':[], 'limit':[], 'grouby_by':[],'having':[], 'single_join':[], 'multiple_join':[], 'multiple_select':[], 'intersect':[], 'except':[], 'union':[]}

  for item in data:
      db_id = item['db_id']
      question = item['question']
      query = sqlparse.format(item['query'], keyword_case='upper')

      if db_id not in db_ids_com_inconsistencia:
        schema = dbs[db_id]
        schemaComEx = dbsComEx[db_id]

        path = db_path + db_id + '/' + db_id + '.sqlite'
        answer = run_db(get_db(path), query) #consultar(db_path + db_id + '/' + db_id + '.sqlite', query)
        if 'Error' in answer[0]:
          print(answer)
        else:
          dataset_dict['db_id'].append(db_id)
          dataset_dict['schema'].append(schema)
          dataset_dict['schemaComEx'].append(schemaComEx)
          dataset_dict['question'].append(question)
          dataset_dict['query'].append(query)
          dataset_dict['answer'].append(str(answer))

          distinct=False
          like=False
          between=False
          order_by=False
          limit=False
          grouby_by=False
          having=False
          single_JOIN=False
          multiple_JOIN=False
          multiple_SELECT=False
          intersect=False
          exceptt=False
          union=False

          toks =  query.split()

          qtd_select = toks.count("SELECT")
          if qtd_select > 1:
            if 'INTERSECT' in toks:
              intersect=True
              print('INTERSECT:', query)
            elif 'EXCEPT' in toks:
              exceptt=True
              print('EXCEPT:', query)
            elif 'UNION' in toks:
              union=True
              print('UNION:', query)
            else:
              multiple_SELECT=True

          if 'JOIN' in toks:
            qtd_join = toks.count("JOIN")
            if  qtd_join > 1:
              multiple_JOIN=True
            else:
              single_JOIN=True

          if 'DISTINCT' in toks:
            distinct=True

          if 'LIKE' in toks:
            like=True

          if 'BETWEEN' in toks:
            between=True

          if 'ORDER' in toks:
             order_by=True

          if 'LIMIT' in toks:
            limit=True

          if 'GROUP' in toks:
            grouby_by=True

          if 'HAVING' in toks:
            having=True

          if (multiple_SELECT):
            complexity = 'Extra Hard'
          elif (multiple_JOIN or intersect or exceptt or union):
            complexity = 'Hard'
          elif (single_JOIN):
            complexity = 'Medium'
          else:
            complexity = 'Easy'

          dataset_dict['complexity'].append(complexity)

          dataset_dict['distinct'].append(distinct)
          dataset_dict['like'].append(like)
          dataset_dict['between'].append(between)
          dataset_dict['order_by'].append(order_by)
          dataset_dict['limit'].append(limit)
          dataset_dict['grouby_by'].append(grouby_by)
          dataset_dict['having'].append(having)
          dataset_dict['single_join'].append(single_JOIN)
          dataset_dict['multiple_join'].append(multiple_JOIN)
          dataset_dict['multiple_select'].append(multiple_SELECT)
          dataset_dict['intersect'].append(intersect)
          dataset_dict['except'].append(exceptt)
          dataset_dict['union'].append(union)

          # numero_de_select = item['query_toks'].count("SELECT")
          # if numero_de_select > 1:
          #   dataset_dict['complexity'].append('Extra Hard') #multiple SELECT
          # else:
          #   sql_info = item['sql']
          #   if len(sql_info['from']['table_units']) > 2:
          #       dataset_dict['complexity'].append('Hard') #multiple JOIN
          #   elif len(sql_info['from']['table_units']) == 2:
          #       dataset_dict['complexity'].append('Medium') #single JOIN
          #   elif sql_info['groupBy']:
          #       dataset_dict['complexity'].append('aggregation')
          #   elif sql_info['orderBy']:
          #       dataset_dict['complexity'].append('order')
          #   else:
          #       dataset_dict['complexity'].append('Easy')

  return dataset_dict

#### Train Dataset

##### databases

In [ ]:
database_train_path = '/content/drive/Shareddrives/LLMs/Datasets/SpiderDataset/database/'

In [ ]:
import os

train_databases = []
conteudo = os.listdir(database_train_path)
# Itera sobre o conteúdo do diretório
for item in conteudo:
    train_databases.append(item)

train_databases = sorted(train_databases)

In [ ]:
train_database_schemas = {}
train_database_schemasComEx = {}
for i in train_databases:

  if i=='baseball_1' or i== 'imdb' or i=='loan_1' or i=='store_product' or i=='restaurants':
    print(f'{i} --> tem insconsitência de foreign key')
    # with open(base_path+i+'/schema.sql', 'r') as f:
    #   sql_lines = f.readlines()

    # for line in sql_lines:
    #   if 'INSERT INTO' not in line:
    #     if 'insert into' not in line:
    #       print(line)
  elif i=='college_1' or i=='flight_1' or i=='music_1' or i=='wta_1':
     print(f'{i} --> Invalid isoformat string')

  else:
    # print(i)
    path = database_train_path + i + '/' + i + '.sqlite'
    train_database_schemas[i] = get_schema(path)
    train_database_schemasComEx[i] = get_schema(path, qtd_ex=2)

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self._metadata.reflect(


baseball_1 --> tem insconsitência de foreign key
college_1 --> Invalid isoformat string
flight_1 --> Invalid isoformat string
imdb --> tem insconsitência de foreign key
loan_1 --> tem insconsitência de foreign key
music_1 --> Invalid isoformat string
restaurants --> tem insconsitência de foreign key


/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('store_id', 'store', 'store_id')' could not be located in PRAGMA foreign_keys for table staff
  self._metadata.reflect(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('Event_ID', 'Events', 'Event_ID')' could not be located in PRAGMA foreign_keys for table Assets_in_Events
  self._metadata.reflect(


store_product --> tem insconsitência de foreign key


/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.BIGINT'> with reflected arguments ['20']; using no arguments.
  self._metadata.reflect(


wta_1 --> Invalid isoformat string


##### json

In [ ]:
db_ids_train_com_inconsistencia = ['baseball_1','imdb','loan_1','store_product','restaurants','college_1','flight_1','music_1','wta_1']

###### train_spider

In [ ]:
train_spider_path = '/content/drive/Shareddrives/LLMs/Datasets/SpiderDataset/train_spider.json'

In [ ]:
train_spider = informacoes_json(train_spider_path,database_train_path,train_database_schemas,train_database_schemasComEx,db_ids_train_com_inconsistencia)

INTERSECT: SELECT T3.born_state FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T1.name  =  'Treasury' INTERSECT SELECT T3.born_state FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T1.name  =  'Homeland Security'
INTERSECT: SELECT Status FROM city WHERE Population  >  1500 INTERSECT SELECT Status FROM city WHERE Population  <  500
INTERSECT: SELECT Status FROM city WHERE Population  >  1500 INTERSECT SELECT Status FROM city WHERE Population  <  500
UNION: SELECT student_id FROM student_course_registrations UNION SELECT student_id FROM student_course_attendance
UNION: SELECT student_id FROM student_course_registrations UNION SELECT student_id FROM student_course_attendance
UNION: SELECT course_id FROM student_course_registrations WHERE student_id = 121 UNION SELECT course_id FROM student_course_attendance WHERE s

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self._metadata.reflect(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.BIGINT'> with reflected arguments ['20']; using no arguments.
  self._metadata.reflect(


INTERSECT: SELECT T2.f1 FROM user_profiles AS T1 JOIN follows AS T2 ON T1.uid  =  T2.f2 WHERE T1.name  =  "Mary" INTERSECT SELECT T2.f1 FROM user_profiles AS T1 JOIN follows AS T2 ON T1.uid  =  T2.f2 WHERE T1.name  =  "Susan"
EXCEPT: SELECT StuID FROM Student EXCEPT SELECT StuID FROM Has_allergy
EXCEPT: SELECT StuID FROM Student EXCEPT SELECT StuID FROM Has_allergy
INTERSECT: SELECT lname ,  age FROM Student WHERE StuID IN (SELECT StuID FROM Has_allergy WHERE Allergy  =  "Milk" INTERSECT SELECT StuID FROM Has_allergy WHERE Allergy  =  "Cat")
INTERSECT: SELECT lname ,  age FROM Student WHERE StuID IN (SELECT StuID FROM Has_allergy WHERE Allergy  =  "Milk" INTERSECT SELECT StuID FROM Has_allergy WHERE Allergy  =  "Cat")
EXCEPT: SELECT fname ,  sex FROM Student WHERE StuID IN (SELECT StuID FROM Has_allergy WHERE Allergy  =  "Milk" EXCEPT SELECT StuID FROM Has_allergy WHERE Allergy  =  "Cat")
EXCEPT: SELECT fname ,  sex FROM Student WHERE StuID IN (SELECT StuID FROM Has_allergy WHERE Aller

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('store_id', 'store', 'store_id')' could not be located in PRAGMA foreign_keys for table staff
  self._metadata.reflect(


INTERSECT: SELECT title ,  film_id FROM film WHERE rental_rate  =  0.99 INTERSECT SELECT T1.title ,  T1.film_id FROM film AS T1 JOIN inventory AS T2 ON T1.film_id  =  T2.film_id GROUP BY T1.film_id HAVING count(*)  <  3
INTERSECT: SELECT title ,  film_id FROM film WHERE rental_rate  =  0.99 INTERSECT SELECT T1.title ,  T1.film_id FROM film AS T1 JOIN inventory AS T2 ON T1.film_id  =  T2.film_id GROUP BY T1.film_id HAVING count(*)  <  3
UNION: SELECT payment_date FROM payment WHERE amount  >  10 UNION SELECT T1.payment_date FROM payment AS T1 JOIN staff AS T2 ON T1.staff_id  =  T2.staff_id WHERE T2.first_name  =  'Elsa'
UNION: SELECT payment_date FROM payment WHERE amount  >  10 UNION SELECT T1.payment_date FROM payment AS T1 JOIN staff AS T2 ON T1.staff_id  =  T2.staff_id WHERE T2.first_name  =  'Elsa'
EXCEPT: SELECT title FROM film WHERE LENGTH  >  100 OR rating  =  'PG' EXCEPT SELECT title FROM film WHERE replacement_cost  >  200
EXCEPT: SELECT title FROM film WHERE LENGTH  >  100 OR

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('Event_ID', 'Events', 'Event_ID')' could not be located in PRAGMA foreign_keys for table Assets_in_Events
  self._metadata.reflect(


INTERSECT: SELECT Product_Type_Code FROM Products WHERE Product_Price  >  4500 INTERSECT SELECT Product_Type_Code FROM Products WHERE Product_Price  <  3000
INTERSECT: SELECT YEAR FROM festival_detail WHERE LOCATION  =  'United States' INTERSECT SELECT YEAR FROM festival_detail WHERE LOCATION != 'United States'
INTERSECT: SELECT T2.customer_name ,  T2.payment_method FROM mailshot_customers AS T1 JOIN customers AS T2 ON T1.customer_id  =  T2.customer_id WHERE T1.outcome_code  =  'Order' INTERSECT SELECT T2.customer_name ,  T2.payment_method FROM mailshot_customers AS T1 JOIN customers AS T2 ON T1.customer_id  =  T2.customer_id WHERE T1.outcome_code  =  'No Response'
EXCEPT: SELECT DName FROM DEPARTMENT EXCEPT SELECT T1.DName FROM DEPARTMENT AS T1 JOIN MINOR_IN AS T2 ON T1.DNO  =  T2.DNO
EXCEPT: SELECT DName FROM DEPARTMENT EXCEPT SELECT T1.DName FROM DEPARTMENT AS T1 JOIN MINOR_IN AS T2 ON T1.DNO  =  T2.DNO
UNION: SELECT CName FROM COURSE WHERE Credits  =  3 UNION SELECT CName FROM COUR

In [ ]:
# train_spider_dataset = Dataset.from_dict(train_spider)
# train_spider_dataset
try:
    train_spider_dataset = Dataset.from_dict(train_spider)
    print(train_spider_dataset)
except Exception as e:
    print(f"Erro ao criar o conjunto de dados: {e}")

Dataset({
    features: ['db_id', 'schema', 'schemaComEx', 'question', 'query', 'answer', 'complexity', 'distinct', 'like', 'between', 'order_by', 'limit', 'grouby_by', 'having', 'single_join', 'multiple_join', 'multiple_select', 'intersect', 'except', 'union'],
    num_rows: 6431
})


###### train_others

In [ ]:
train_others_path = '/content/drive/Shareddrives/LLMs/Datasets/SpiderDataset/train_others.json'

In [ ]:
train_others = informacoes_json(train_others_path,database_train_path,train_database_schemas,train_database_schemasComEx,db_ids_train_com_inconsistencia)

In [ ]:
train_others_dataset = Dataset.from_dict(train_others)
train_others_dataset

Dataset({
    features: ['db_id', 'schema', 'schemaComEx', 'question', 'query', 'answer', 'complexity', 'distinct', 'like', 'between', 'order_by', 'limit', 'grouby_by', 'having', 'single_join', 'multiple_join', 'multiple_select', 'intersect', 'except', 'union'],
    num_rows: 1425
})

###### dev

In [ ]:
train_dev_path = '/content/drive/Shareddrives/LLMs/Datasets/SpiderDataset/dev.json'

In [ ]:
train_dev = informacoes_json(train_dev_path,database_train_path,train_database_schemas,train_database_schemasComEx,db_ids_train_com_inconsistencia)

INTERSECT: SELECT country FROM singer WHERE age  >  40 INTERSECT SELECT country FROM singer WHERE age  <  30
EXCEPT: SELECT name FROM stadium EXCEPT SELECT T2.name FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  =  2014
EXCEPT: SELECT name FROM stadium EXCEPT SELECT T2.name FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  =  2014
INTERSECT: SELECT T2.name ,  T2.location FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.Year  =  2014 INTERSECT SELECT T2.name ,  T2.location FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.Year  =  2015
INTERSECT: SELECT T2.name ,  T2.location FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.Year  =  2014 INTERSECT SELECT T2.name ,  T2.location FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.Year  =  2015
INTERSECT: SELECT t1.fname FROM student AS 

In [ ]:
train_dev_dataset = Dataset.from_dict(train_dev)
train_dev_dataset

Dataset({
    features: ['db_id', 'schema', 'schemaComEx', 'question', 'query', 'answer', 'complexity', 'distinct', 'like', 'between', 'order_by', 'limit', 'grouby_by', 'having', 'single_join', 'multiple_join', 'multiple_select', 'intersect', 'except', 'union'],
    num_rows: 972
})

##### train (concatenacao)

In [ ]:
from datasets import concatenate_datasets

train_combined = concatenate_datasets([train_spider_dataset, train_others_dataset, train_dev_dataset])

In [ ]:
train_combined

Dataset({
    features: ['db_id', 'schema', 'schemaComEx', 'question', 'query', 'answer', 'complexity', 'distinct', 'like', 'between', 'order_by', 'limit', 'grouby_by', 'having', 'single_join', 'multiple_join', 'multiple_select', 'intersect', 'except', 'union'],
    num_rows: 8828
})

In [ ]:
train_combined.push_to_hub('spider', token='', split = 'train')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/lleticiasilvaa/spider/commit/1bbae983c7acbcaf21ba0cb4c774cd092b3fa217', commit_message='Upload dataset', commit_description='', oid='1bbae983c7acbcaf21ba0cb4c774cd092b3fa217', pr_url=None, pr_revision=None, pr_num=None)

#### Test Dataset

##### databases

In [ ]:
database_test_path = '/content/drive/Shareddrives/LLMs/Datasets/SpiderDataset/test_database/'

In [ ]:
import os

test_databases = []
conteudo = os.listdir(database_test_path)
# Itera sobre o conteúdo do diretório
for item in conteudo:
    test_databases.append(item)

test_databases = sorted(test_databases)

In [ ]:
test_database_schemas = {}
test_database_schemasComEx = {}
for i in test_databases:
  # print(i)

  if i=='baseball_1' or i=='book_1' or i=='car_racing' or i== 'imdb' or i=='loan_1' or i=='restaurants' or i=='store_product':
    print(f'{i} --> tem insconsitência de foreign key')
    # with open(base_path+i+'/schema.sql', 'r') as f:
    #   sql_lines = f.readlines()

    # for line in sql_lines:
    #   if 'INSERT INTO' not in line:
    #     if 'insert into' not in line:
    #       print(line)

  elif i=='college_1' or i=='cre_Students_Information_Systems' or i=='flight_1' or i=='music_1' or i=='planet_1' or i=='wta_1':
     print(f'{i} --> Invalid isoformat string')

  else:
    path = database_test_path + i + '/' + i + '.sqlite'
    test_database_schemas[i] = get_schema(path)
    test_database_schemasComEx[i] = get_schema(path,qtd_ex=2)

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self._metadata.reflect(


baseball_1 --> tem insconsitência de foreign key
book_1 --> tem insconsitência de foreign key
car_racing --> tem insconsitência de foreign key
college_1 --> Invalid isoformat string
cre_Students_Information_Systems --> Invalid isoformat string
flight_1 --> Invalid isoformat string
imdb --> tem insconsitência de foreign key
loan_1 --> tem insconsitência de foreign key
music_1 --> Invalid isoformat string
planet_1 --> Invalid isoformat string
restaurants --> tem insconsitência de foreign key


/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('store_id', 'store', 'store_id')' could not be located in PRAGMA foreign_keys for table staff
  self._metadata.reflect(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: WARNING: SQL-parsed foreign key constraint '('Event_ID', 'Events', 'Event_ID')' could not be located in PRAGMA foreign_keys for table Assets_in_Events
  self._metadata.reflect(


store_product --> tem insconsitência de foreign key


/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/sql_database.py:122: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.BIGINT'> with reflected arguments ['20']; using no arguments.
  self._metadata.reflect(


wta_1 --> Invalid isoformat string


##### json

In [ ]:
db_ids_test_com_inconsistencia = ['baseball_1','book_1','car_racing', 'imdb','loan_1','restaurants','store_product','college_1','cre_Students_Information_Systems','flight_1','music_1','planet_1','wta_1']

In [ ]:
test_path = '/content/drive/Shareddrives/LLMs/Datasets/SpiderDataset/test_data/dev.json'

In [ ]:
test= informacoes_json(test_path, database_test_path, test_database_schemas, test_database_schemasComEx, db_ids_test_com_inconsistencia)

INTERSECT: SELECT Country FROM player WHERE Earnings  >  1400000 INTERSECT SELECT Country FROM player WHERE Earnings  <  1100000
INTERSECT: SELECT Country FROM player WHERE Earnings  >  1400000 INTERSECT SELECT Country FROM player WHERE Earnings  <  1100000
INTERSECT: SELECT T1.customer_id ,  T1.customer_first_name ,  T1.customer_last_name FROM Customers AS T1 JOIN Orders AS T2 ON T1.customer_id  =  T2.customer_id GROUP BY T1.customer_id HAVING count(*)  >  2 INTERSECT SELECT T1.customer_id ,  T1.customer_first_name ,  T1.customer_last_name FROM Customers AS T1 JOIN Orders AS T2 ON T1.customer_id  =  T2.customer_id JOIN Order_items AS T3 ON T2.order_id  =  T3.order_id GROUP BY T1.customer_id HAVING count(*)  >= 3
INTERSECT: SELECT T1.customer_id ,  T1.customer_first_name ,  T1.customer_last_name FROM Customers AS T1 JOIN Orders AS T2 ON T1.customer_id  =  T2.customer_id GROUP BY T1.customer_id HAVING count(*)  >  2 INTERSECT SELECT T1.customer_id ,  T1.customer_first_name ,  T1.custome

In [ ]:
test_dataset = Dataset.from_dict(test)
test_dataset

Dataset({
    features: ['db_id', 'schema', 'schemaComEx', 'question', 'query', 'answer', 'complexity', 'distinct', 'like', 'between', 'order_by', 'limit', 'grouby_by', 'having', 'single_join', 'multiple_join', 'multiple_select', 'intersect', 'except', 'union'],
    num_rows: 1867
})

In [ ]:
test_dataset.push_to_hub('spider', token='', split = 'test')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/lleticiasilvaa/spider/commit/e67cb87b4d9185e03db86691dc43cb465826189c', commit_message='Upload dataset', commit_description='', oid='e67cb87b4d9185e03db86691dc43cb465826189c', pr_url=None, pr_revision=None, pr_num=None)

### Colocar na Organização

In [ ]:
dataset = load_dataset("lleticiasilvaa/spider")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1867 [00:00<?, ? examples/s]

In [ ]:
dataset.push_to_hub("NESPED-GEN/spider", token='')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/NESPED-GEN/spider/commit/7631271514712dd547dc7ec53b74e172ef0eb907', commit_message='Upload dataset', commit_description='', oid='7631271514712dd547dc7ec53b74e172ef0eb907', pr_url=None, pr_revision=None, pr_num=None)